# Advent of Code Day 23
#### Problem in full can be found here: https://adventofcode.com/2023/day/23

In [85]:
# The input is too big to use it fully, so instead of using the map, create a graph that will add the
# possible nodes to the graph then recursively find the longest path using the graph rather than the map
# Part 1: has slopes '>' '<' 'v' '^' where it forces one direction
# Part 2: remove the slopes

# Get the input in a map
file = open("inputday23.txt")
map = []
for line in file:
    line = line.strip()
    map.append(list(line))
file.close()

# Get height, width of grid
height, width = len(grid), len(grid[0])

# Update the starting and end point of grid to not be on the edge
map[0][1] = '#'
map[height - 1][width - 2] = '#'
start = (1, 1)
end = (height - 2, width - 2)

# Get the graph and find the longest path and add 2 to it because 2 points were removed above
graph = createGraph(grid, start, end)
print(findHikes(graph, start, end) + 2)
# Update the graph to ignore slopes and find the next
g = graph_from_grid(grid, src, dst, ignore_slopes=True)
print(longest_path(g, src, dst) + 2)

2358
6586


In [46]:
from collections import defaultdict, deque

def createGraph(map, current, end, ignore_slopes=False):
    # Initialize graph, queue, seen
	g = defaultdict(list)
	q = deque([src])
	seen = set()

    # While the queue is not empty
	while q:
        # Pop the row column of the queue
		rc = q.popleft() # (row, column)
        # If the row column has been seen already, go next
		if rc in seen:
			continue

        # Add row column to seen
		seen.add(rc)

        # Find the adjacent nodes on the current row column
		for n, dist in adjacent_nodes(map, rc, current, end, ignore_slopes): # (r, c), distance
            # Add the node to the graph
			g[rc].append((n, dist))
            # Add the row, column to the queue
			q.append(n)

    # Return the graph
	return g

In [69]:
def adjacent_nodes(map, rc, current, end, ignore_slopes):
    # Initialize queue, seen
    q = deque([(rc, 0)])
    seen = set()
    
    # While the queue is not empty
    while q:
        # Pop the row column and distance from source
        rc, dist = q.popleft()
        # Add to seen
        seen.add(rc)
        
        # Collect the possible paths
        possiblePaths = findPaths(map, rc, ignore_slopes)
        for n in neighbors(grid, *rc, ignore_slopes): # n = (r, c)
            # If the path has been visited already, go to the next
            if n in seen:
                continue
            
            # If the path is a node, return it to the graph
            if is_node(map, n, current, end, ignore_slopes, possiblePaths):
                # Yield returns the (r, c), distance + 1 but then continues to execute the function still
                yield (n, dist + 1)
                continue
            # Add the next path to the queue
            q.append((n, dist + 1))

In [77]:
def is_node(map, rc, current, end, ignore_slopes, possiblePaths):
    # Return true if the current row, column is the source, destination or has more than 2 possible moves
    return rc == current or rc == end or num_neighbors(grid, *rc, ignore_slopes) > 2

In [50]:
# A function that finds a path and returns the max length of steps for the path
def findHikes(graph, current, end, distance=0, seen=set()):  
    # If the destination is reached, return the distance
	if current == end:
		return distance

    # Initialize the best distance and seen
	best = 0
	seen.add(current)

    # For each neighbor and distance in the graph
	for neighbor, weight in graph[current]:
        # If the neighbour has already been visited, go to the next
		if neighbor in seen:
			continue

        # Recursively find the best by taking the max of the current best and the neighbor best
		best = max(best, longest_path(graph, neighbor, end, distance + weight))

    # Remove current from the set
	seen.remove(current)
    # Return the best distance (largest)
	return best

In [83]:
def neighbors(grid, r, c, ignore_slopes):
    # Get current symbol
	cell = grid[r][c]

    # Yield allows it to be more memory efficient and calculate each possible path 1 by 1

    # If the path ignores slopes or is a '.' then take into account all 4 directions
	if ignore_slopes or cell == '.':
		for r, c in ((r + 1, c), (r - 1, c), (r, c + 1), (r, c - 1)):
			if grid[r][c] != '#':
				yield r, c
    # Else only take into account one direction depending on the symbol
	elif cell == 'v': yield (r + 1, c)
	elif cell == '^': yield (r - 1, c)
	elif cell == '>': yield (r, c + 1)
	elif cell == '<': yield (r, c - 1)

In [82]:
def num_neighbors(grid, r, c, ignore_slopes):
    # Return the total amount of possible moves
	if ignore_slopes or grid[r][c] == '.':
		return sum(grid[r][c] != '#' for r, c in ((r + 1, c), (r - 1, c), (r, c + 1), (r, c - 1)))
	return 1

In [23]:
import sys
sys.setrecursionlimit(100000)